# Clean and Organize data

In [83]:
import os

class bball_pipeline(object):

    def make_folder(self):
        if not os.path.exists('./data'):
            os.mkdir('./data')
    
    def save_file(self):
        pass


In [84]:
# our imports for the model
import pandas as pd
import numpy as np

In [85]:
# ncaa tournament results into dataframe 
ncaa_short = pd.read_csv('ncaa_short.csv')  
# team ID, name, sports reference url name
team_data = pd.read_csv('team_list.csv')  
# Season long stats for 2009-2018
season_data = pd.read_csv('season_data.csv')
# more team data
season_stats2 = pd.read_csv('more_team_data.csv')

In [86]:
#team_data.head()
ncaa_short.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2009,134,1287,58,1106,43,N,0
1,2009,136,1163,103,1151,47,N,0
2,2009,136,1181,86,1127,62,N,0
3,2009,136,1211,77,1103,64,N,0
4,2009,136,1261,75,1139,71,N,0


# Create the 'teams' dataframe

This will be done in the following steps:
1. create an empty dataframe with the desired column titles.
2. Iterate through the ncaa_short dataframe's rows. These are the past ncaa_tournament games. 
3. For each game, filter the 'teams' dataframe that we are building, to the rows which include the correct year
4. If the schools are not in the filtered 'teams' dataframe, add them, and create a new team_id.


In [87]:
# create the dataframe
teams = pd.DataFrame(columns = ["team_id" , "school_id" , "year"] ) 
# counter 
id_counter = 1000000

# iterate through our data
for i in range(ncaa_short.shape[0]):
    year = ncaa_short.iloc[i , 0] 
    id_1 =  ncaa_short.iloc[i , 2]
    id_2 =  ncaa_short.iloc[i , 4] 
    s = teams["year"] == year              # filter
    if  id_1 not in s:                     # conditionally append
        teams = teams.append({'team_id': id_counter , 'school_id': id_1 , "year" : year} , ignore_index=True )
        id_counter += 1  # update counter
    if  id_2 not in s:                     # conditionally append
        teams = teams.append({'team_id': id_counter , 'school_id': id_2 , "year" : year} , ignore_index=True )    
        id_counter += 1


In [88]:
teams.head()

,team_id,school_id,year
0,1000000,1287,2009
1,1000001,1106,2009
2,1000002,1163,2009
3,1000003,1151,2009
4,1000004,1181,2009


Let's first get a sense of what our data looks like

In [4]:
# Get the columns
season_stats2.columns

Index(['Rk', 'Season', 'Conf', 'W', 'L', 'W-L%', 'W.1', 'L.1', 'W-L%.1', 'SRS',
       'SOS', 'PTS', 'PTS.1', 'AP Pre', 'AP High', 'AP Final',
       'NCAA Tournament', 'Seed', 'Coach(es)', 'Unnamed: 19', 'team'],
      dtype='object')

Remove some non helpful datavals

In [5]:
season_stats2 = season_stats2.drop(columns=['Rk', 'Unnamed: 19' , 'AP Pre' , 'AP High' , 'W.1' , 'L.1' , 'W-L%.1',  'AP Final' , 'NCAA Tournament'])

In [6]:
season_stats2.describe(include = 'all')

,Season,Conf,W,L,W-L%,SRS,SOS,PTS,PTS.1,Seed,Coach(es),team
count,2600,2600,2600,2600,2600.000000,2591.000000,2591.000000,2599.000000,2599.000000,736.000000,2600,2600
unique,17,35,50,42,NaN,NaN,NaN,NaN,NaN,NaN,2488,217
top,2018-19,ACC,20,14,NaN,NaN,NaN,NaN,NaN,NaN,Sean Miller (27-8),north-carolina-greensboro
freq,217,165,162,221,NaN,NaN,NaN,NaN,NaN,NaN,3,12
mean,NaN,NaN,NaN,NaN,0.564665,3.337522,1.442721,71.233282,68.418853,8.728261,NaN,NaN
std,NaN,NaN,NaN,NaN,0.165013,9.661552,5.510206,5.602585,5.266216,4.662143,NaN,NaN
min,NaN,NaN,NaN,NaN,0.050000,-30.400000,-12.710000,52.600000,50.700000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,0.454250,-3.680000,-3.110000,67.400000,64.800000,5.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,0.576000,3.310000,1.040000,71.300000,68.300000,9.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,0.686000,10.655000,6.565000,75.000000,71.900000,13.000000,NaN,NaN


In [7]:
season_stats2.dtypes

Season        object
Conf          object
W             object
L             object
W-L%         float64
SRS          float64
SOS          float64
PTS          float64
PTS.1        float64
Seed         float64
Coach(es)     object
team          object
dtype: object

In [8]:
# have to get rid of the * ( this means the data was changed as a result of a penalty)
season_stats2['W'] = season_stats2['W'].map(lambda x: x.rstrip('*'))
season_stats2['L'] = season_stats2['L'].map(lambda x: x.rstrip('*'))

In [9]:
season_stats2 = season_stats2.astype({'W': 'int32' , 'L': 'int32'})

In [10]:
season_stats2.dtypes

Season        object
Conf          object
W              int32
L              int32
W-L%         float64
SRS          float64
SOS          float64
PTS          float64
PTS.1        float64
Seed         float64
Coach(es)     object
team          object
dtype: object